In [1]:
using Distributed

In [2]:
addprocs(2)

2-element Array{Int64,1}:
 2
 3

In [3]:
@everywhere using DistributedArrays, DistributedJets, Jets, JetPackWave, ParallelOperations, Schedulers, Wave

In [4]:
v = read!("../marmousi_20m_nz151_nx461.bin", Array{Float32}(undef, 151,461));
dz,dx = 0.02,0.02
nz,nx = size(v)
@show dz,dx
@show nz,nx;

(dz, dx) = (0.02, 0.02)
(nz, nx) = (151, 461)


In [5]:
sx = (0.0:dx*50:(461-1)*0.02)
nshots = length(sx)
@show nshots;

nshots = 10


## Broadcast the velocity (v) to workers

In [6]:
_v = bcast(v)

ArrayFutures with pids=[2, 3, 1] and type (151, 461)

## Build a list of keyword arguments to be passed to the modeling operator

In [9]:
kwargs = (reportinterval = 0, freqQ=5, srcfieldfile="", nsponge = 200)

(reportinterval = 0, freqQ = 5, srcfieldfile = "", nsponge = 200)

## Define function to be run by the Scheduler

In [12]:
@everywhere function modelshot(isrc,sx,_v;kwargs...)
    nz,nx,dz,dx = 151,461,0.02,0.02    
    @info "modeling shot $(isrc) on $(gethostname()) with id $(myid())..."
    F = JopNlProp2DAcoIsoDenQ_DEO2_FDTD(;
        b = ones(Float32,nz,nx),
        isinterior = true,
        ntrec = 1101,
        dtrec = 0.002,
        dtmod = 0.001,
        dz = dz,
        dx = dx,
        wavelet = WaveletCausalRicker(f=5.0),
        sx = sx[isrc],
        sz = dz,
        rx = dx*[0:0.5:nx-1;],
        rz = 2*dz*ones(length(0:0.5:nx-1)),
        kwargs...)
    
    d = F*_v
    @info "...done modeling shot $(isrc) on $(gethostname()) with id $(myid())"
    @info "extrema of shot $(isrc) is $(extrema(d))"
    nothing
end

## Use cvxpmap to schedule tasks to workers (dynamic schedule)

In [13]:
cvxpmap(i->modelshot(i,sx,localpart(_v);kwargs...), 1:nshots)

[ Info: modeling shot 2 on sal-gate0352.gcn0.chevron.net with id 3...
[ Info: modeling shot 1 on sal-gate0352.gcn0.chevron.net with id 2...
[ Info: ...done modeling shot 2 on sal-gate0352.gcn0.chevron.net with id 3
[ Info: extrema of shot 2 is (-6.126741f-5, 0.00011243532f0)
[ Info: modeling shot 3 on sal-gate0352.gcn0.chevron.net with id 3...
[ Info: ...done modeling shot 1 on sal-gate0352.gcn0.chevron.net with id 2
[ Info: extrema of shot 1 is (-6.1267005f-5, 0.000112433096f0)
[ Info: modeling shot 4 on sal-gate0352.gcn0.chevron.net with id 2...
[ Info: ...done modeling shot 3 on sal-gate0352.gcn0.chevron.net with id 3
[ Info: extrema of shot 3 is (-6.147483f-5, 0.00011341105f0)
[ Info: modeling shot 5 on sal-gate0352.gcn0.chevron.net with id 3...
[ Info: ...done modeling shot 4 on sal-gate0352.gcn0.chevron.net with id 2
[ Info: extrema of shot 4 is (-6.059044f-5, 0.00011074651f0)
[ Info: modeling shot 6 on sal-gate0352.gcn0.chevron.net with id 2...
[ Info: ...done modeling shot 5 on

## Remove workers

In [14]:
rmprocs(workers())

Task (done) @0x00002adfb5ce8760